In [4]:
import numpy as np
from numpy import ma
import math
import scipy
import os
from IPython.display import Image

from netCDF4 import Dataset

from cartopy import config
import cartopy.crs as ccrs
from cartopy.mpl.ticker import LongitudeFormatter, LatitudeFormatter
from cartopy.util import add_cyclic_point

import matplotlib.pyplot as plt
import matplotlib.colors as colors
from matplotlib.cm import get_cmap
from matplotlib import ticker

# from matplotlib.mlab import bivariate_normal

NA = 6.022e+23

In [7]:
# 2*2.5

FILEDIR1 = '/n/home12/hongwei/HONGWEI/merra2_2x25_standard/'
NcFile1   = Dataset(FILEDIR1+'/GEOSChem.SpeciesConc_inst.20150101_0000z.nc4','r',format='NETCDF4_CLASSIC')

lat1             = NcFile1.variables['lat'][:]
lon1             = NcFile1.variables['lon'][:]
EU1              = NcFile1.variables['SpeciesConc_PASV_EU']
LA1              = NcFile1.variables['SpeciesConc_PASV_LA']

GC_AREA1 = NcFile1.variables['AREA']


# 4*5
FILEDIR2 = '/n/home12/hongwei/HONGWEI/merra2_4x5_standard_2/'
NcFile2   = Dataset(FILEDIR2+'/GEOSChem.SpeciesConc_inst.20150101_0000z.nc4','r',format='NETCDF4_CLASSIC')

lat2             = NcFile2.variables['lat'][:]
lon2             = NcFile2.variables['lon'][:]
EU2              = NcFile2.variables['SpeciesConc_PASV_EU']
LA2              = NcFile2.variables['SpeciesConc_PASV_LA']

GC_AREA2 = NcFile2.variables['AREA']


LA1,'',LA2

RuntimeError: NetCDF: HDF error

In [ ]:
#------------------------------------------------
# total air mass in each grid  ------------------
#------------------------------------------------

AD_file1 = open(FILEDIR1+'State_Met_AD_backup.txt','r')

GC_AD1 = EU1[0,:,:,:]*0.0

Nx1 = len(EU1[0,0,0,:])
Ny1 = len(EU1[0,0,:,0])
Nz1 = len(EU1[0,:,0,0])
Nt1 = len(EU1[:,0,0,0])

for ix in range(Nx1):
    for iy in range(Ny1):
        for iz in range(Nz1):
            line = AD_file1.readline()
            GC_AD1[iz,iy,ix] = float(line)
            
            
AD_file2 = open(FILEDIR2+'State_Met_AD_backup.txt','r')

GC_AD2 = EU2[0,:,:,:]*0.0

Nx2 = len(EU2[0,0,0,:])
Ny2 = len(EU2[0,0,:,0])
Nz2 = len(EU2[0,:,0,0])
Nt2 = len(EU2[:,0,0,0])

for ix in range(Nx2):
    for iy in range(Ny2):
        for iz in range(Nz2):
            line = AD_file2.readline()
            GC_AD2[iz,iy,ix] = float(line)

In [ ]:
### change unit, do average

EU1_molec = EU1[:,:,:,:]*0.0
for i in range(Nt1):
    EU1_molec[i,:,:,:] = EU1[i,:,:,:]*(GC_AD1[:,:,:]*1000.0/28.97)*NA # [mol/mol] to [molec]
    

EU1_Zsum = np.sum(EU1_molec[:,:,:,:], axis=1)
for i in range(Nt1):
    EU1_Zsum[i,:,:] = EU1_Zsum[i,:,:]/GC_AREA1[:,:]/1e4 # [molec/cm2]

EU1_Zsum_Xmean = np.mean(EU1_Zsum[:,:,:], axis=2)



LA1_molec = LA1[:,:,:,:]*0.0
for i in range(Nt1):
    LA1_molec[i,:,:,:] = LA1[i,:,:,:]*(GC_AD1[:,:,:]*1000.0/28.97)*NA


LA1_Zsum = np.sum(LA1_molec[:,:,:,:], axis=1)
for i in range(Nt1):
    LA1_Zsum[i,:,:]   = LA1_Zsum[i,:,:]/GC_AREA1[:,:]/1e4

LA1_Zsum_Xmean = np.mean(LA1_Zsum[:,:,:], axis=2)


In [ ]:
EU2_molec = EU2[:,:,:,:]*0.0
for i in range(Nt2):
    EU2_molec[i,:,:,:] = EU2[i,:,:,:]*(GC_AD2[:,:,:]*1000.0/28.97)*NA # [mol/mol] to [molec]
    

EU2_Zsum = np.sum(EU2_molec[:,:,:,:], axis=1)
for i in range(Nt2):
    EU2_Zsum[i,:,:] = EU2_Zsum[i,:,:]/GC_AREA2[:,:]/1e4 # [molec/cm2]

EU2_Zsum_Xmean = np.mean(EU2_Zsum[:,:,:], axis=2)



LA2_molec = LA2[:,:,:,:]*0.0
for i in range(Nt2):
    LA2_molec[i,:,:,:] = LA2[i,:,:,:]*(GC_AD2[:,:,:]*1000.0/28.97)*NA


LA2_Zsum = np.sum(LA2_molec[:,:,:,:], axis=1)
for i in range(Nt2):
    LA2_Zsum[i,:,:]   = LA2_Zsum[i,:,:]/GC_AREA2[:,:]/1e4

LA2_Zsum_Xmean = np.mean(LA2_Zsum[:,:,:], axis=2)

In [ ]:
# delete original variable to save memory
del EU1, LA1, EU2, LA2

In [ ]:


t = Nt1-1
print(f'Total mass [molec] in the end: ')
print(f'GEOS-Chem         = {np.sum(EU1_molec[t,:,:,:])}')
print(f'plume model 2*2.5 = {np.sum(LA1_molec[t,:,:,:])}')
print(f'plume model 4*5   = {np.sum(LA2_molec[t,:,:,:])}')

LA1_mass = [0]
LA2_mass = [0]
EU1_mass = [0]
EU2_mass = [0]
days = [0]

for i in range(Nt1):
    days.append(i+1)
    LA1_mass.append(np.sum(LA1_molec[i,:,:,:]))
    LA2_mass.append(np.sum(LA2_molec[i,:,:,:]))
    EU1_mass.append(np.sum(EU1_molec[i,:,:,:]))
    EU2_mass.append(np.sum(EU2_molec[i,:,:,:]))

plt.plot(days, LA1_mass, 'r', label='coupled GEOS-Chem with Plume in 2*2.5');
plt.plot(days, LA2_mass, 'b', label='coupled GEOS-Chem with Plume in 4*5');

plt.plot(days, EU1_mass, 'k', label='conventional GEOS-Chem in 2*2.5');
plt.plot(days, EU2_mass, 'y.', label='conventional GEOS-Chem in 4*5');

plt.title('Injected inert tracer in GEOS-Chem grid');
plt.xlabel('day');
plt.ylabel('Total mass [molec]');
plt.legend();

In [3]:
# plot  -----------------------------------------
#------------------------------------------------

# time step for ploting is 24 hours (once every day)
print(Nt1)

X_max = np.max(LA1_Zsum_Xmean[:,:],axis=(0,1)) # 16.5e+11;

i=0
while i<Nt1:
    print(i)
    fig = plt.figure(figsize=(16,8));
#     gs = fig.add_gridspec(15, 8)
    plt.suptitle('Day: '+str(i+1), fontsize=16);
    
    
    
    # (1) for Lagrange ============================================================
    projection=ccrs.PlateCarree()
    ax1 = plt.subplot(2, 2, 1, projection=ccrs.PlateCarree());
    ax1.set_global()
    
    ax1.gridlines(x_inline=True)    
    ax1.set_xticks(np.linspace(-180, 180, 7), crs=projection)
    ax1.set_yticks(np.linspace(-90, 90, 7), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax1.xaxis.set_major_formatter(lon_formatter)
    ax1.yaxis.set_major_formatter(lat_formatter)
    

#     z = ma.masked_where(LA_Zsum[i,:,:] <= 1e-10, LA_Zsum[i,:,:])

    levels = [0.01e12, 0.05e12, 0.1e12, 0.5e12, 1e12, 5e12, 10e12]
    norm = colors.BoundaryNorm(boundaries=levels, ncolors=256)
    
    data = LA1_Zsum[i,:,:]
    cyclic_data = add_cyclic_point(data)
    data, lon1_cyclic = add_cyclic_point(data, coord=lon1)
    
    ax1_contours = ax1.contourf(lon1_cyclic, lat1, data, levels=levels, norm=norm,\
                                transform=ccrs.PlateCarree(), cmap=get_cmap("Reds"), extend="max");
    
    cb1 = plt.colorbar(ax1_contours, ax=ax1)
    cb1.ax.tick_params(labelsize=11)
    
    ax1.coastlines();
    ax1.set_title('Lagrangian (2*2.5)', fontsize=11);


    
    # (2) for Lagrange Distribution: ==============================================
    ax2 = plt.subplot(2,2,2);
    
    ax2.plot(LA1_Zsum_Xmean[i,:], lat1, 'r-', label='2*2.5');
    ax2.plot(LA2_Zsum_Xmean[i,:], lat2, 'b--', label='4*5');


    ax2.set_xlim(0,X_max);
    ax2.set_ylim(-90,90);
    ax2.set_ylabel('Latitude (deg)');
    ax2.tick_params(axis='x', labelsize=11)  # to Set Matplotlib Tick Labels Font Size
    ax2.tick_params(axis='y', labelsize=11)
    ax2.legend();
    ax2.set_title('Lagrangian', fontsize=11);
    
    
    
     # (3) for EU ==================================================================
    ax3 = plt.subplot(2, 2, 3, projection=ccrs.PlateCarree());
    ax3.set_global()
    
    ax3.gridlines(x_inline=True)    
    ax3.set_xticks(np.linspace(-180, 180, 7), crs=projection)
    ax3.set_yticks(np.linspace(-90, 90, 7), crs=projection)
    lon_formatter = LongitudeFormatter(zero_direction_label=True)
    lat_formatter = LatitudeFormatter()
    ax3.xaxis.set_major_formatter(lon_formatter)
    ax3.yaxis.set_major_formatter(lat_formatter)
    
#     z = ma.masked_where(EU_Zsum[i,:,:] <= 1e-5, EU_Zsum[i,:,:])
    data = EU1_Zsum[i,:,:]
    cyclic_data = add_cyclic_point(data)
    data, lon1_cyclic = add_cyclic_point(data, coord=lon1)
    
    ax3_contours = ax3.contourf(lon1_cyclic, lat1, data, levels=levels, norm=norm,\
                                transform=ccrs.PlateCarree(), cmap=get_cmap("Reds"), extend="max");


    cb3 = plt.colorbar(ax3_contours, ax=ax3)
    cb3.ax.tick_params(labelsize=11)
    
    ax3.coastlines();
    ax3.set_title('Eulerian (2*2.5)', fontsize=11);
    
    
    
    # (4) for GOES distribution ===================================================
    
    ax4 = plt.subplot(2,2,4);
    
    ax4.plot(EU1_Zsum_Xmean[i,:], lat1, 'r-', label='2*2.5');
    ax4.plot(EU2_Zsum_Xmean[i,:], lat2, 'b--', label='4*5');

    ax4.set_xlim(0,X_max);
    ax4.set_ylim(-90,90);
    ax4.set_xlabel('Tracer Concentration [molec/cm2]');
    ax4.set_ylabel('Latitude (deg)');
    ax4.tick_params(axis='x', labelsize=11)  # to Set Matplotlib Tick Labels Font Size
    ax4.tick_params(axis='y', labelsize=11)
    ax4.legend()
    ax4.set_title('Eulerian', fontsize=11);
    
    
    plt.tight_layout()
    plt.savefig(str(i+1)+'_xy2.png', transparent=False, facecolor='w', edgecolor='w');
    plt.close();
    plt.clf();
    plt.cla();
    
    i = i + 1;
    
plt.close();
Image(filename=f'{Nt1-1}_xy2.png') 

NameError: name 'Nt1' is not defined

In [13]:
os.system("./png2gif2.sh")

0